# EDR Transfer

With this notebook you can create an offer for use of an API (Provider) as well as negotiate and receive a access token for such an existing offer (Consumer) using the Management API of the Connector.

Documentation for all endpoints of the Connectors Management API can be found on [SwaggerHub](https://app.swaggerhub.com/apis/eclipse-tractusx-bot/tractusx-edc/0.6.0#/).

## Preperation

Here some values are initialized that will be necessary for the other steps.


### Set up

Import the requests module and assing the base url of the Dataspace as a variable.

In [2]:
from pprint import pprint
import re
import requests

base_url = "https://dataspace-dev-dataspace.base-x-ecosystem.org"

### Fill in Values

Fill in the values for the variables below.

In [3]:
username = "..."
password = "..."
connector_name = "..."

### Get token

Gets the token for accessing your Connector. This token needs to be passed in the header of each request to your Connector.

In [ ]:
url = f"{base_url}/backend/token"
payload = {
  "username": username,
  "password": password
}
response = requests.post(url, json=payload)
token = response.json()["access_token"]

response.raise_for_status()
print("Got token:\n")
print(re.sub(f"(.{{{100}}})", r"\1" + "\n", token).rstrip("\n"))

token_header = {"Authorization": f"Bearer {token}"}

## Provider

Here you as a provider offer access of an API in the Dataspace.

### Create Asset

Here you create an Asset for access of the API you want to offer. The Asset on one hand describes what kind of data is offered (properties) and on the other hand how the Connector can eventually make requests to (dataAddress). However, this will be not be available for others as an Offer just yet.

Choose a unique id for your Asset and specify the file from your storage you want to offer.

In [8]:
asset_id = "..."
api_url = "https://echo.free.beeceptor.com"

Creates an Asset with the given values.

In [ ]:
url = f"{base_url}/backend/connectors/{connector_name}/management/v3/assets"
payload = {
  "@context": {},
  "@id": asset_id,
  "properties": {
    "name": f"API at {api_url}",
    "description": "EDR test"
  },
  "dataAddress": {
    "@type": "DataAddress",
    "type": "HttpData",
    "baseUrl": api_url,
    "proxyMethod": "true",
    "proxyBody": "true"
  }
}

response = requests.post(url, json=payload, headers=token_header)
response.raise_for_status()
print(f"Created Asset with ID: {asset_id}")

### Create Policy

Here you create a Policy which is basically a collection of terms and conditions. This will later be "assigned" to the Asset in order to make it available for others given they meet the conditions.

Choose a unique id for your Policy.

In [10]:
policy_id = "..."

policy = {
  "@type": "odrl:Set",
  "odrl:permission": [
    {
      "odrl:action": "USE"
    }
  ]
}

Creates a Policy with the given values.

In [ ]:
url = f"{base_url}/backend/connectors/{connector_name}/management/v3/policydefinitions"
payload = {
  "@context": {
    "odrl": "http://www.w3.org/ns/odrl/2/"
  },
  "@id": policy_id,
  "policy": policy
}

response = requests.post(url, json=payload, headers=token_header)
response.raise_for_status()
print(f"Created Policy with ID: {policy_id}")

### Create Contract Definition

Here you create a Contract whose role it is to assign Policies to Assets. The contract's "Access Policy" sets the terms and conditions under which the Asset will be visible as an Offer in the Catalog and the "Contract Policy" sets the terms and conditions under which the Negotiation for an Offer will be agreed. The "Assets Selector" determines to which Assets the Policies are "assigned" to.

Choose a unique id for your Policy.

In [12]:
contract_id = "..."

Creates a Contract with the given values.

In [ ]:
url = f"{base_url}/backend/connectors/{connector_name}/management/v2/contractdefinitions"
payload = {
  "@context": {
    "odrl": "http://www.w3.org/ns/odrl/2/"
  },
  "@id": contract_id,
  "accessPolicyId": policy_id,
  "contractPolicyId": policy_id,
  "assetsSelector": {
    "operandLeft": "https://w3id.org/edc/v0.0.1/ns/id",
    "operator": "=",
    "operandRight": asset_id
  }
}

response = requests.post(url, json=payload, headers=token_header)
response.raise_for_status()
print(f"Created Contract Definition with ID: {contract_id}")

If everything was successful the Asset will be now available to others as an Offer in your Catalog.

## Consumer

Here you as a consumer negotiate for an Offer in the Federated Catalog (the collection of all participant's catalogs) and then send requests to the associated API.

### Get Federated Catalog

Here you get the contents of the Federated Catalog and then choose an Offer you want to negotiate for.

Gets and prints out the relevant conents of the Federated Catalog.

In [ ]:
url = f"{base_url}/backend/federated/catalog/query"
payload = {
  "@context": {},
  "@type": "QuerySpec"
}

response = requests.post(url, json=payload, headers=token_header)
response.raise_for_status()
print("Got the federated catalog")

federated_catalog = response.json()
for catalog in federated_catalog:
  print(f"Catalog from {catalog['originator']}")
  datasets = catalog["dcat:dataset"]
  if not isinstance(datasets, list):
    datasets = [datasets]
  for dataset in datasets:
    if "name" not in dataset:
      continue
    print(f"    Offer for {dataset['name']}")

Pick out an Offer from the printed ones and fill in the below values for the variables accordingly.

In [25]:
catalog_from = "..."
offer_for = "..."

Gets the necessary data about the chosen Offer from the Federated Catalog in order to start a Negotiation and an eventual Transfer in the next steps.

In [ ]:
for catalog in federated_catalog:
  if catalog["originator"] != catalog_from:
    continue
  datasets = catalog["dcat:dataset"]
  if not isinstance(datasets, list):
    datasets = [datasets]
  for dataset in datasets:
    if not "name" in dataset or dataset["name"] != offer_for:
      continue
    provider_id = catalog["dspace:participantId"]
    originator = catalog["originator"]
    policy = dataset["odrl:hasPolicy"]
    offered_asset_id = dataset["id"]

print("Got necessary values\n")

print(f"ProviderId:  {provider_id}")
print(f"Originator: {originator}")
print(f"Policy with ID: {policy['@id']}")
print(f"AssetId: {offered_asset_id}")

### Initiate EDR Negotiation

Here you start an EDR Negotiation for the Offer chosen in the previous step. This performs the usual Negotiation and Tranfer in such a way that you can later retrieve access tokens. If the given conditions are satisfied the EDR Negotiation will succeed and an Agreement will be created.

Starts the EDR Negotiation using the values got from the previous step.

In [ ]:
url = f"{base_url}/backend/connectors/{connector_name}/management/v3/edrs"
payload = {
	"@context": {
		"odrl": "http://www.w3.org/ns/odrl/2/"
	},
	"counterPartyAddress": originator,
	"protocol": "dataspace-protocol-http",
	"policy": policy | {"odrl:assigner": {"@id": provider_id}, "odrl:target": {"@id": offered_asset_id}}
}

response = requests.post(url, json=payload, headers=token_header)
response.raise_for_status()

negotiation_id = response.json()["@id"]
print(f"Started EDR Negotiation with ID: {negotiation_id}")

As mentioned the EDR starts a Negotiation and a Transfer. Information about the Negotiation can be easily accessed by the below request since we know its ID from the previous step. However, for the transfer we do not know the ID so it is not as easy to get information on it at this point.

In [ ]:
url = f"{base_url}/backend/connectors/{connector_name}/management/v2/contractnegotiations/{negotiation_id}"

response = requests.get(url, headers=token_header)
response.raise_for_status()
print(f"Negotiation data:\n")
pprint(response.json())

## Get EDR

Here you will get the access token.

Gets all EDRs for the chosen asset.

In [ ]:
url = f"{base_url}/backend/connectors/{connector_name}/management/v3/edrs/request"
payload = {
  "@context": {},
  "@type": "QuerySpec",
  "filterExpression": [
        {
            "operandLeft": "assetId",
            "operator": "=",
            "operandRight": offered_asset_id
        }
        
    ]
}

response = requests.post(url, json=payload, headers=token_header)
response.raise_for_status()

print(f"EDRs data:\n")
pprint(response.json())

transfer_id = response.json()[0]["transferProcessId"]

Uses the Transfer Id from the previous step to get the acccess token.

In [ ]:
url = f"{base_url}/backend/connectors/{connector_name}/management/v3/edrs/{transfer_id}/dataaddress"

response = requests.get(url, headers=token_header)
response.raise_for_status()

print(f"EDR data:\n")
pprint(response.json())

access_token = response.json()["authorization"]
endpoint = response.json()["endpoint"]

# Access the API

Now you can use the access token to make requests to the offered API. Your request will go to the provider's Connector which will confirm the existence and validity of the associated Agreeement and then reroute your request to the API. Finally the Connector will send back the response of the API.

In [ ]:
headers = {"Authorization": access_token}

payload = {
    "test": "This is a test."
}

endpoint = f"{base_url}/backend/connectors/alice/public"
response = requests.post(endpoint, headers=headers, json=payload)

print("Response:")
pprint(response.json())